In [1]:
import pandas as pd
data_path = "../../data/open_source_8454_combine_short_description.csv"  # 替換為你的檔案路徑
df = pd.read_csv(data_path)
# df_label_0 = df[df["Label"] == 0]
# df_label_0_to_drop = df_label_0.sample(n=3000, random_state=42)
# df = df.drop(df_label_0_to_drop.index)
df

,Inbound Message,Label
0,login issue verified user details employee man...,0
1,outlook hello team meetings skype meetings etc...,0
2,cant log vpn cannot log vpn best,0
3,unable access tool page unable access tool page,0
4,skype error skype error,0
...,...,...
8449,emails coming mail good afternoon receiving em...,22
8450,telephony software issue telephony software issue,0
8451,vip windows password reset tifpdchb pedxruyf v...,0
8452,machine est funcionando unable access machine ...,44


# 十折驗證法

In [2]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}") 
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
print(f"Current GPU Memory Allocated: {torch.cuda.memory_allocated() / 1e9} GB")
print(f"Current GPU Memory Cached: {torch.cuda.memory_reserved() / 1e9} GB")

Using device: cuda
CUDA Available: True
CUDA Device: NVIDIA GeForce RTX 3050 6GB Laptop GPU
Current GPU Memory Allocated: 0.0 GB
Current GPU Memory Cached: 0.0 GB


In [3]:
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, messages, labels, tokenizer, max_length):
        self.messages = messages
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.messages)

    def __getitem__(self, idx):
        message = str(self.messages[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            message,
            max_length=self.max_length,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [4]:
# 定義訓練與評估函數
def train_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0

    for batch in tqdm(dataloader):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
    
    return total_loss / len(dataloader)

In [5]:
from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef
from sklearn.preprocessing import label_binarize
import numpy as np
import torch

def eval_model(model, dataloader, criterion, num_labels):
    model.eval()
    total_loss = 0
    all_labels = []
    all_preds = []
    all_probs = []
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels)
            total_loss += loss.item()

            probs = torch.softmax(logits, dim=1)
            _, preds = torch.max(logits, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    val_loss = total_loss / len(dataloader)
    val_accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    balanced_acc = balanced_accuracy_score(all_labels, all_preds)
    mcc = matthews_corrcoef(all_labels, all_preds)

    return {
        "val_loss": val_loss,
        "val_accuracy": val_accuracy,
        "precision": precision,
        "recall": recall,
        "f1-score": f1,
        "macro_f1-score": macro_f1,
        "balanced_accuracy": balanced_acc,
        "mcc": mcc
    }

In [6]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import KFold
# 設定參數
MAX_LENGTH = 128
BATCH_SIZE = 16
EPOCHS = 20

# 初始化 BERT 的 tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 初始化數據
messages = df['Inbound Message'].tolist()
labels = df['Label'].tolist()

# KFold 初始化
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [7]:
from torch.utils.data import DataLoader
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from tqdm import tqdm
import numpy as np

all_fold_results = []

# 進行 10 折交叉驗證
for fold, (train_idx, val_idx) in enumerate(kf.split(messages)):
    print(f"Fold {fold + 1}")
    
    train_messages = [messages[i] for i in train_idx]
    train_labels = [labels[i] for i in train_idx]
    val_messages = [messages[i] for i in val_idx]
    val_labels = [labels[i] for i in val_idx]

    print("Training set label distribution:", {label: train_labels.count(label) for label in set(train_labels)})
    print("Validation set label distribution:", {label: val_labels.count(label) for label in set(val_labels)})
    
    train_dataset = CustomDataset(train_messages, train_labels, tokenizer, MAX_LENGTH)
    val_dataset = CustomDataset(val_messages, val_labels, tokenizer, MAX_LENGTH)
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    NUM_LABELS = len(set(train_labels) | set(val_labels))
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=NUM_LABELS)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=1e-2)
    criterion = CrossEntropyLoss()

    epoch_results = []
    patience = 3
    best_macro_f1 = 0
    epochs_without_improvement = 0
    
    for epoch in range(EPOCHS):
        print(f"Epoch {epoch + 1}/{EPOCHS}")
        train_loss = train_epoch(model, train_dataloader, optimizer, criterion)
        metrics = eval_model(model, val_dataloader, criterion, num_labels=NUM_LABELS)
        epoch_results.append(metrics)
        print(metrics)

        current_macro_f1 = metrics["macro_f1-score"]
        
        if current_macro_f1 > best_macro_f1:
            best_macro_f1 = current_macro_f1  # 更新最佳 macro_f1-score
            epochs_without_improvement = 0   # 重置計數
        else:
            epochs_without_improvement += 1  # 增加計數
        
        # 若連續 patience 3 次沒有提升，則停止訓練
        if epochs_without_improvement >= patience:
            print(f"Early stopping triggered at epoch {epoch + 1}")
            break
    
    avg_metrics = {metric: np.mean([epoch[metric] for epoch in epoch_results]) for metric in epoch_results[0].keys()}
    all_fold_results.append(avg_metrics)

final_metrics = {metric: np.mean([result[metric] for result in all_fold_results]) for metric in all_fold_results[0].keys()}
print("Final 10-fold Cross-Validation Results:")
for metric, value in final_metrics.items():
    print(f"{metric}: {value}")

Fold 1
Training set label distribution: {0: 3585, 1: 28, 2: 123, 3: 26, 4: 239, 5: 134, 6: 101, 7: 37, 8: 80, 9: 70, 10: 82, 11: 199, 12: 209, 13: 34, 14: 25, 15: 31, 16: 22, 17: 258, 18: 106, 19: 47, 20: 15, 21: 39, 22: 89, 23: 183, 24: 15, 25: 55, 26: 100, 27: 52, 28: 13, 29: 15, 30: 17, 31: 89, 32: 39, 33: 36, 34: 29, 35: 13, 36: 32, 37: 24, 38: 5, 39: 118, 40: 10, 41: 10, 42: 166, 43: 18, 44: 23, 45: 11, 46: 58, 47: 601, 48: 215, 49: 82}
Validation set label distribution: {0: 389, 1: 3, 2: 17, 3: 4, 4: 18, 5: 11, 6: 17, 7: 2, 8: 5, 9: 11, 10: 6, 11: 16, 12: 32, 13: 2, 14: 4, 16: 3, 17: 31, 18: 10, 19: 9, 20: 3, 21: 5, 22: 8, 23: 17, 24: 5, 25: 8, 26: 7, 27: 9, 28: 2, 29: 1, 30: 2, 31: 11, 32: 6, 33: 4, 34: 8, 35: 2, 36: 3, 37: 3, 38: 2, 39: 11, 40: 4, 41: 1, 42: 18, 43: 2, 44: 2, 46: 10, 47: 60, 48: 37, 49: 5}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20


100%|██████████| 476/476 [03:22<00:00,  2.35it/s]


{'val_loss': 1.5968256626489028, 'val_accuracy': 0.5945626477541371, 'precision': 0.45092886753169803, 'recall': 0.5945626477541371, 'f1-score': 0.5063891504880338, 'macro_f1-score': 0.11891580474464518, 'balanced_accuracy': 0.14033472496793956, 'mcc': 0.4392452156060018}
Epoch 2/20


...

{'val_loss': 1.69317235361855, 'val_accuracy': 0.6686390532544378, 'precision': 0.6750325030852319, 'recall': 0.6686390532544378, 'f1-score': 0.6450548217916814, 'macro_f1-score': 0.40174398472351736, 'balanced_accuracy': 0.40782684375529826, 'mcc': 0.5727554021328068}
Epoch 12/20


100%|██████████| 476/476 [03:21<00:00,  2.36it/s]


{'val_loss': 1.7425469109472238, 'val_accuracy': 0.6745562130177515, 'precision': 0.6626356679680732, 'recall': 0.6745562130177515, 'f1-score': 0.6486129028419073, 'macro_f1-score': 0.40140657876290103, 'balanced_accuracy': 0.39110361712421593, 'mcc': 0.5791039439942566}
Epoch 13/20


100%|██████████| 476/476 [03:21<00:00,  2.36it/s]


{'val_loss': 1.6864866664949454, 'val_accuracy': 0.6686390532544378, 'precision': 0.6525660344326576, 'recall': 0.6686390532544378, 'f1-score': 0.6416122042831169, 'macro_f1-score': 0.39748714361637194, 'balanced_accuracy': 0.38902687306672595, 'mcc': 0.5695075022867868}
Early stopping triggered at epoch 13
Final 10-fold Cross-Validation Results:
val_loss: 1.455616617184091
val_accuracy: 0.6684718540839654
precision: 0.6507668102779423
recall: 0.6684718540839654
f1-score: 0.6409466515373194
macro_f1-score: 0.32504084310531406
balanced_accuracy: 0.3398207185489883
mcc: 0.5551576820954109


In [9]:
all_fold_results

[{'val_loss': 1.578385956266766,
  'val_accuracy': 0.658628841607565,
  'precision': 0.6503274406650326,
  'recall': 0.658628841607565,
  'f1-score': 0.6352906121201572,
  'macro_f1-score': 0.3428556073307238,
  'balanced_accuracy': 0.34840062251705517,
  'mcc': 0.5488094271790669},
 {'val_loss': 1.4481227529592484,
  'val_accuracy': 0.6719858156028368,
  'precision': 0.6481105175442193,
  'recall': 0.6719858156028368,
  'f1-score': 0.6415100893940947,
  'macro_f1-score': 0.3028108671958957,
  'balanced_accuracy': 0.31779347728612467,
  'mcc': 0.5521923965949521},
 {'val_loss': 1.3624264196233051,
  'val_accuracy': 0.6836091410559496,
  'precision': 0.6649101357775477,
  'recall': 0.6836091410559496,
  'f1-score': 0.6540342185605859,
  'macro_f1-score': 0.2902588721785395,
  'balanced_accuracy': 0.30233946962161545,
  'mcc': 0.5601299861479693},
 {'val_loss': 1.374023511532956,
  'val_accuracy': 0.6795615731785944,
  'precision': 0.6658766101748623,
  'recall': 0.6795615731785944,
  'f